# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察標籤編碼與獨編碼熱的影響

# [作業重點]
- 回答在範例中的觀察結果
- 觀察標籤編碼與獨熱編碼, 在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，  
對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?

In [17]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'D:/100days/data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)
=>ANS:  獨熱編碼 + 羅吉斯迴歸 score >標籤編碼 + 羅吉斯迴歸
1.標籤編碼 + 羅吉斯迴歸
shape : (891, 5)
score : 0.780004837244799
time : 0.018948793411254883 sec
2.獨熱編碼 + 羅吉斯迴歸
shape : (891, 2429)
score : 0.8013346043513216
time : 0.16025161743164062 sec

In [19]:
# 標籤編碼 + 羅吉斯迴歸

df_temp = pd.DataFrame()
print('df_temp:',df_temp)
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
    print(df_temp[c])
train_X = df_temp[:train_num]
print(train_X )
estimator = LogisticRegression()
start = time.time()
print(start) 
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

df_temp: Empty DataFrame
Columns: []
Index: []
0        155
1        286
2        523
3        422
4         22
5        818
6        767
7        914
8        605
9        847
10      1067
11       133
12      1070
13        38
14      1221
15       533
16       995
17      1271
18      1217
19       760
20       423
21       112
22       778
23      1111
24       917
25        64
26       371
27       404
28       885
29      1186
        ... 
1279     192
1280     915
1281     934
1282     715
1283       1
1284     437
1285     654
1286    1119
1287     246
1288     417
1289     683
1290     259
1291     132
1292     424
1293     434
1294     206
1295     413
1296     872
1297    1229
1298    1260
1299    1006
1300     936
1301     849
1302     801
1303     527
1304    1126
1305     891
1306    1046
1307    1227
1308     953
Name: Name, Length: 1309, dtype: int64
0       1
1       0
2       0
3       0
4       1
5       1
6       1
7       1
8       0
9       0
10      0
11      0
1

In [20]:
# 獨熱編碼 + 羅吉斯迴歸

df_temp = pd.get_dummies(df)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 2429)
score : 0.8013346043513216
time : 0.16025161743164062 sec
